In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col,broadcast,lead
class Transformer:
    def __init__(self):
        pass;
    def transform(self, inputDFs):
        pass;
class AirpodsAfterIphoneTransformer(Transformer):
    def transform(self, inputDFs):
        """
            Customer who have bought Airpods after buying the iphone
        """
        transactionInputDF = inputDFs.get("transactionInputDF")
        print("transactionInputDF in transform")
        transactionInputDF.show()

        windowSpec = Window.partitionBy("customer_id").orderBy("transaction_date")

        tranformedDf = transactionInputDF.withColumn("Next Product Name",lead('product_name').over(windowSpec))

        print("Airpods after buying iphone")
        tranformedDf.orderBy("customer_id","transaction_date","product_name").show()

        filteredDF = tranformedDf.filter(
            (col("product_name") == "iPhone") & (col("Next Product Name") == "AirPods")
        )
        print("FILTERED DF")
        filteredDF.orderBy("customer_id","transaction_date","product_name").show()

        customerInputDF = inputDFs.get("customerInputDF")

        customerInputDF.show()

        joinDF = customerInputDF.join(
            broadcast(filteredDF),
            "customer_id"
        )

        print("JOINED DF")
        joinDF.show()

        return joinDF.select(
            'customer_id',
            "customer_name",
            "location"
        )

class OnlyAirpodsAndIphoneTransformer(Transformer):

    def transform(self, inputDFs): 
        """
        Customer Who have Bought only iPhone and Airpods nothing else
        """
        transactionInputDF = inputDFs.get("transactionInputDF")
        print("transactionInputDF in transform")
        groupDF = transactionInputDF.groupBy("customer_id").agg(
            collect_set("product_name").alias("products")
        )
        groupDF.show()


        filteredDF = groupDF.filter(
            (array_contains(col("products"), "iPhone")) &
            (array_contains(col("products"), "AirPods")) & 
            (size(col("products")) == 2)
        )
        print("Only Airpods and iPhone")
        filteredDF.show()


        customerInputDF = inputDFs.get("customerInputDF")

        customerInputDF.show()

        joinDF = customerInputDF.join(
            broadcast(filteredDF),
            "customer_id"
        )

        print("JOINED DF")
        joinDF.show()

        return joinDF.select(
            'customer_id',
            "customer_name",
            "location"
        )


        



